In [95]:
# These scripts are used to parse my data into

import pandas as pd
import numpy as np
import os

target_dir = "Bike 07-06-23 8:52"
target_dir2 = "Bike 07-06-23 14:27"

In [89]:
from statistics import mean

def time_diff(df):
    res = []
    for i in range(len(df["Time (s)"]) - 1):
        val1 = df["Time (s)"][i]
        val2 = df["Time (s)"][i + 1]
        
        res.append(val2 - val1)
    return mean(res)

def read_phyphox(parent_dir):
    acc = pd.read_csv(os.path.join(parent_dir, "Phyphox", "Accelerometer.csv"))
    gyro = pd.read_csv(os.path.join(parent_dir, "Phyphox", "Gyroscope.csv"))
    loc = pd.read_csv(os.path.join(parent_dir, "Phyphox", "Location.csv"))
        
    data_len = min(len(acc), len(gyro))
    acc = acc[0:data_len]
    gyro = gyro[0:data_len]
        
    time_step = mean([time_diff(acc), time_diff(gyro)])
    
    for i in range(data_len):
        time = time_step * i
        acc["Time (s)"][i] = time
        gyro["Time (s)"][i] = time
    
    acc.set_index('Time (s)', inplace=True)
    gyro.set_index('Time (s)', inplace=True)
    loc.set_index('Time (s)', inplace=True)
        
            
    merged = acc.join(gyro, how="outer")
    merged = pd.concat([merged, loc]).sort_index().interpolate()
    
    # Rename columns
    merged.index.names = ["time"]
    merged.rename(inplace=True, columns={
        "Acceleration x (m/s^2)": "acceleration_x",
        "Acceleration y (m/s^2)": "acceleration_y",
        "Acceleration z (m/s^2)": "acceleration_z",
        "Gyroscope x (rad/s)": "gyroscope_x",
        "Gyroscope y (rad/s)": "gyroscope_y",
        "Gyroscope z (rad/s)": "gyroscope_z",
        "Latitude (°)": "latitude",
        "Longitude (°)": "longitude",
        "Height (m)": "height",
        "Velocity (m/s)": "velocity",
        "Direction (°)": "direction",
        "Horizontal Accuracy (m)": "h_accuracy",
        "Vertical Accuracy (m)": "v_accuracy",

    })
    merged = merged.dropna()
    
    time_df = pd.read_csv(os.path.join(parent_dir, "Phyphox", "meta", "time.csv"))
    start_time = time_df.loc[time_df["event"] == "START"]["system time"][0]
    
    merged.reset_index(inplace=True)
    merged['time'] = pd.to_datetime(merged['time'] + start_time,unit='s')
    return merged

read_phyphox(target_dir)

,time,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,latitude,longitude,height,velocity,direction,h_accuracy,v_accuracy
0,2023-06-07 06:22:51.237444864,1.479975,4.688025,8.505450,-0.083256,-0.035956,-0.047644,52.278535,4.824785,-67.529845,0.758701,64.440002,9.10,91.699997
1,2023-06-07 06:22:51.357940992,0.751950,3.868050,9.076950,-0.041250,-0.046475,-0.062150,52.278468,4.824891,-69.529879,0.701824,58.365002,7.05,89.599998
2,2023-06-07 06:22:51.751848960,1.387300,4.062050,8.925950,-0.119212,0.199558,-0.065450,52.278402,4.824997,-71.529913,0.644947,52.290001,5.00,87.500000
3,2023-06-07 06:22:52.240046080,2.022650,4.256050,8.774950,-0.197175,0.445592,-0.068750,52.278388,4.825015,-74.229921,1.020694,24.639999,3.50,85.500000
4,2023-06-07 06:22:52.357411328,2.658000,4.450050,8.623950,-0.275137,0.691625,-0.072050,52.278392,4.825022,-74.629912,0.946574,24.105000,3.40,82.450001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568,2023-06-07 06:52:37.411697664,6.423000,-10.324051,1.669050,0.150287,-0.078237,-0.280775,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000
3569,2023-06-07 06:52:38.411168000,13.989000,-12.897000,-1.638000,1.387787,3.768462,-3.214887,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000
3570,2023-06-07 06:52:39.410638592,0.193950,3.823950,9.574950,-0.194150,0.261525,-0.985875,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000
3571,2023-06-07 06:52:40.410108928,0.586050,4.098000,7.506001,-0.145200,0.679525,0.005225,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000


In [77]:
def read_samsung_health(parent_dir):
    files = os.scandir(os.path.join(parent_dir, "SamsungHealth"))
    filename = next(filter(lambda file: "com.samsung.health.exercise.live_data.json" in file.name, files)).name
        
    df = pd.read_json(os.path.join(parent_dir, "SamsungHealth", filename))
    df = df[["start_time", "heart_rate"]].dropna()

    return df

read_samsung_health(target_dir)

,start_time,heart_rate
1,2023-06-07 06:22:31.971,97.0
3,2023-06-07 06:22:41.550,90.0
5,2023-06-07 06:22:43.547,89.0
6,2023-06-07 06:22:51.541,100.0
8,2023-06-07 06:23:01.628,101.0
...,...,...
419,2023-06-07 06:52:32.054,154.0
420,2023-06-07 06:52:42.056,153.0
421,2023-06-07 06:52:52.057,157.0
422,2023-06-07 06:53:01.531,154.0


In [102]:
def read_combined(parent_dir):
    pp = read_phyphox(parent_dir)
    sh = read_samsung_health(parent_dir)
    print(sh)
    
    pp["heart_rate"] = np.nan
    
    
    # Finds the time intervals of the heart rate measurements and update the phyphox heart rate accordingly
    # This is done as samsung measurements are not very fine grained and only give use relatively large
    # time intervals
    for i in range(len(sh) - 1):
        row1 = sh.iloc[i]
        row2 = sh.iloc[i + 1]
        pp.loc[(pp["time"] >= row1["start_time"]) & (pp["time"] < row2["start_time"]), "heart_rate"] = row1["heart_rate"]
    
    return pp

read_combined(target_dir)

                 start_time  heart_rate
1   2023-06-07 06:22:31.971        97.0
3   2023-06-07 06:22:41.550        90.0
5   2023-06-07 06:22:43.547        89.0
6   2023-06-07 06:22:51.541       100.0
8   2023-06-07 06:23:01.628       101.0
..                      ...         ...
419 2023-06-07 06:52:32.054       154.0
420 2023-06-07 06:52:42.056       153.0
421 2023-06-07 06:52:52.057       157.0
422 2023-06-07 06:53:01.531       154.0
423 2023-06-07 06:53:11.536       141.0

[160 rows x 2 columns]


,time,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,latitude,longitude,height,velocity,direction,h_accuracy,v_accuracy,heart_rate
0,2023-06-07 06:22:51.237444864,1.479975,4.688025,8.505450,-0.083256,-0.035956,-0.047644,52.278535,4.824785,-67.529845,0.758701,64.440002,9.10,91.699997,89.0
1,2023-06-07 06:22:51.357940992,0.751950,3.868050,9.076950,-0.041250,-0.046475,-0.062150,52.278468,4.824891,-69.529879,0.701824,58.365002,7.05,89.599998,89.0
2,2023-06-07 06:22:51.751848960,1.387300,4.062050,8.925950,-0.119212,0.199558,-0.065450,52.278402,4.824997,-71.529913,0.644947,52.290001,5.00,87.500000,100.0
3,2023-06-07 06:22:52.240046080,2.022650,4.256050,8.774950,-0.197175,0.445592,-0.068750,52.278388,4.825015,-74.229921,1.020694,24.639999,3.50,85.500000,100.0
4,2023-06-07 06:22:52.357411328,2.658000,4.450050,8.623950,-0.275137,0.691625,-0.072050,52.278392,4.825022,-74.629912,0.946574,24.105000,3.40,82.450001,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568,2023-06-07 06:52:37.411697664,6.423000,-10.324051,1.669050,0.150287,-0.078237,-0.280775,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000,154.0
3569,2023-06-07 06:52:38.411168000,13.989000,-12.897000,-1.638000,1.387787,3.768462,-3.214887,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000,154.0
3570,2023-06-07 06:52:39.410638592,0.193950,3.823950,9.574950,-0.194150,0.261525,-0.985875,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000,154.0
3571,2023-06-07 06:52:40.410108928,0.586050,4.098000,7.506001,-0.145200,0.679525,0.005225,52.334605,4.864740,-33.228580,2.067639,134.669998,5.90,13.000000,154.0
